In this notebook, me and my team, we are going to scrape an ecommerce website. We'll get into each individual product page and retrieve our information from there. This is the website we are going to scrape <a href="https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt">Cdiscount</a>  – it's an French online shop.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Maintenant, nous allons définir l'URL de base de la page principale car nous en aurons besoin lorsque nous construirons nos URL pour chacun des produits individuels.

De plus, nous enverrons un agent utilisateur sur chaque requête HTTP, car si nous avons effectuez une requête GET à l'aide de requests , l'agent utilisateur est Python par défaut, ce qui peut être bloqué.

Donc, pour remplacer cela, nous allons déclarer une variable qui stockera notre user-agent.

In [ ]:
baseurl = "https://www.cdiscount.com/telephonie/v-144-0.html#nav=T%C3%A9l%C3%A9phonie_txt"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

In [ ]:
k = requests.get(baseurl, headers=headers).text
soup=BeautifulSoup(k,'html.parser')
productlist = soup.find_all("li",{"class":"cPdtItem jsPdtItem"})
soup


In [ ]:
r = requests.get(baseurl)

#if you want the whole content you can just do slicing stuff on the response stored in r or rather just do it with bs4

soup = BeautifulSoup(r.text, "html.parser")

In [ ]:
from selectorlib import Extractor
import requests 
import json 
from time import sleep

In [ ]:
baseurl="https://www.amazon.com/dp/B07YDGHDFQ/ref=sspa_dk_detail_1?psc=1&pd_rd_i=B07YDGHDFQ&pd_rd_w=V8Vij&pf_rd_p=0c758152-61cd-452f-97a6-17f070f654b8&pd_rd_wg=xTOAJ&pf_rd_r=P8H8N568MSXRHR7H4DRH&pd_rd_r=bb5f00a6-6961-4b95-8810-dbb98dff2669&s=apparel&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzVERWRVJHUkRNOEpNJmVuY3J5cHRlZElkPUEwNDQyOTIxM0JaODZZMFI4SjhFOCZlbmNyeXB0ZWRBZElkPUEwMTE3NDc3MUhKRUZONFJVTzRaUSZ3aWRnZXROYW1lPXNwX2RldGFpbCZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU="

In [ ]:
# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('selectors.yml')
def scrape(url):  

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    # Pass the HTML of the page and create 
    print(r.text)
    return e.extract(r.text)

In [ ]:
baseurl="https://www.amazon.com/Cass-Creations-iPhone-Bumper-Protective/dp/B096ZCT3N3/ref=sr_1_14_sspa?pd_rd_r=30305d08-93c6-4f70-aca0-5bdcd0619f0a&pd_rd_w=V3rlu&pd_rd_wg=cnq1i&pf_rd_p=f5c158e1-98f7-4998-94b8-d7306c066086&pf_rd_r=Q0RJ0DQSNB5EM8EWZX6G&qid=1649545449&refinements=p_72%3A2491149011&s=wireless&sr=1-14-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyMlpGS1o2MU1UNE5EJmVuY3J5cHRlZElkPUEwMzI2MjEyTU1QNVJHU1NDVlpVJmVuY3J5cHRlZEFkSWQ9QTA4MjMxNzAxTlVYRjRSVTBTQlpRJndpZGdldE5hbWU9c3BfbXRmX2Jyb3dzZSZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU="

In [ ]:
from ast import literal_eval
import pandas as pd
 
with open(r'D:\INPT2\Data mining\DataMining-Project\output.json') as f:
    data = f.readlines()
    data = [literal_eval(x.strip()) for x in data]

print(data)
pd.DataFrame(data)

In [ ]:
from venv import create
import requests
from bs4 import BeautifulSoup


def scrape_url_page(url):
    session = requests.Session()
    adapter = requests.adapters.HTTPAdapter(max_retries=20)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    pg=session.get(url)

    html_soup=BeautifulSoup(pg.text,'html.parser')
    url_current=html_soup.findAll(class_="s-pagination-item s-pagination-selected")

    print(html_soup)
    print(url_current)

url="https://www.amazon.com/s?i=mobile&rh=n%3A2335752011%2Cp_72%3A2491149011&page=12&pd_rd_r=30305d08-93c6-4f70-aca0-5bdcd0619f0a&pd_rd_w=V3rlu&pd_rd_wg=cnq1i&pf_rd_p=f5c158e1-98f7-4998-94b8-d7306c066086&pf_rd_r=Q0RJ0DQSNB5EM8EWZX6G&qid=1649612117&ref=sr_pg_12"
scrape_url_page(url)

In [19]:
from venv import create
from click import open_file
import requests
from bs4 import BeautifulSoup


def scrape_url_prod(url):
    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    pg=requests.get(url, headers=headers)

    html_soup=BeautifulSoup(pg.text,'html.parser')
    #print(html_soup)
    with open(r"D:\INPT2\Data mining\DataMining-Project\web Scraping\urls_prod.txt", "a") as urls_prod:
        for i in html_soup.find_all("a",class_="a-link-normal s-no-outline", href=True):
            url_prod="\n"+"https://www.amazon.com"+i["href"]
            print(url_prod)
            urls_prod.write(url_prod)

url="https://www.amazon.com/s?i=mobile&rh=n%3A2335752011%2Cp_72%3A2491149011&page=2&pd_rd_r=30305d08-93c6-4f70-aca0-5bdcd0619f0a&pd_rd_w=V3rlu&pd_rd_wg=cnq1i&pf_rd_p=f5c158e1-98f7-4998-94b8-d7306c066086&pf_rd_r=Q0RJ0DQSNB5EM8EWZX6G&qid=1649829234&ref=sr_pg_2"
scrape_url_prod(url)



https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_browse_mobile_sr_pg2_1?ie=UTF8&adId=A015613130N6SIMMCVAHH&url=%2FArmor-Protective-Magnetic-Carriage-Huawei%2Fdp%2FB07KVM6DPM%2Fref%3Dsr_1_25_sspa%3Fpd_rd_r%3D30305d08-93c6-4f70-aca0-5bdcd0619f0a%26pd_rd_w%3DV3rlu%26pd_rd_wg%3Dcnq1i%26pf_rd_p%3Df5c158e1-98f7-4998-94b8-d7306c066086%26pf_rd_r%3DQ0RJ0DQSNB5EM8EWZX6G%26qid%3D1649914532%26refinements%3Dp_72%253A2491149011%26s%3Dwireless%26sr%3D1-25-spons%26psc%3D1&qualifier=1649914532&id=1549410304052793&widgetName=sp_atf_browse

https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_browse_mobile_sr_pg2_1?ie=UTF8&adId=A06339292JZ2235R4ZXA9&url=%2FCaliCase-Floating-Universal-Waterproof-Pouch%2Fdp%2FB01E7I3M8M%2Fref%3Dsr_1_26_sspa%3Fpd_rd_r%3D30305d08-93c6-4f70-aca0-5bdcd0619f0a%26pd_rd_w%3DV3rlu%26pd_rd_wg%3Dcnq1i%26pf_rd_p%3Df5c158e1-98f7-4998-94b8-d7306c066086%26pf_rd_r%3DQ0RJ0DQSNB5EM8EWZX6G%26qid%3D1649914532%26refinements%3Dp_72%253A2491149011%26s%